<a href="https://colab.research.google.com/github/hadi-M/DS-Unit-2-Applied-Modeling/blob/master/module3-permutation-boosting/HADI_MODARES_LS_DS_233_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1- Imports

In [0]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

from os import getcwd, remove
# from pdb import set_trace as st
from ipdb import set_trace as st

from time import sleep

from glob import glob

from zipfile import ZipFile

import pandas as pd

from IPython.display import display
pd.options.display.max_columns = None

# 2- Retreiving data

In [0]:
def initate_browser():
    # config for now showing the download popup and setting the download dir
    
    # url = "https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=258"
    
#     data for US carriers, demosetic and international
    url = "https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=308"
#     profile = webdriver.FirefoxProfile()
    profile.set_preference('browser.download.folderList', 2) # custom location
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.download.manager.showAlertOnComplete", False)
    profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/zip, text/html, application/x-zip-compressed')
    profile.set_preference("browser.helperApps.neverAsk.openFile",'application/zip, text/html, application/x-zip-compressed')
    profile.set_preference('browser.download.dir', getcwd())

    browser = webdriver.Firefox(profile)
    browser.get(url)
    
    #set AllVars's tickbox
    browser.find_element_by_css_selector('#AllVars').click()

    return browser

In [0]:
def download_file_year(browser, year):
    select = Select(browser.find_element_by_css_selector('#XYEAR'))
    select.select_by_value(str(year))
    browser.find_element_by_css_selector('.tsbutton').click()

In [0]:
def check_if_file_are_downloaded():
    return len(glob("*.zip")) == len(range(1990, 2020))

In [0]:
def check_if_final_csv_is_made():
    return  bool(glob("data.csv"))

In [0]:
def download_files():
    browser = initate_browser()
    for year in range(1990, 2020):
        download_file_year(browser, year)
        sleep(10)
# download_file_year(browser, 2001)


In [0]:
def concatinate_files():
    list_of_dfs = []
    for file_name in glob("*.zip"):
        zipObj = ZipFile(file_name, "r")
        zipObj.extractall()
        csv_file_name = glob("*.csv")[0]
        temp_df = pd.read_csv(csv_file_name)
        list_of_dfs.append(temp_df)
        remove(csv_file_name)
#         print("z")

    full_df = pd.concat(list_of_dfs, axis=0)
    full_df.to_csv("data.csv")

In [0]:
def remove_extra_zip_files():
    for file in glob("*.zip"):
        remove(file)

In [0]:
def get_data():
    # check if final concatinated data.csv is already made
    if not check_if_final_csv_is_made():
        # check if zip files are downloaded
        if not check_if_file_are_downloaded():
            initate_browser()
            download_files()
        concatinate_files()
        remove_extra_zip_files()

In [0]:
get_data()

# 3- Cleaning data

## 3-1 selecting columns, correcting dtypes, removing nulls

In [0]:
if "df" not in locals():
    df = pd.read_csv("./data.csv")

In [0]:
needed_columns = [
    "UNIQUE_CARRIER_NAME",
    "CARRIER_NAME",
    "YEAR",
    "MONTH",
    "QUARTER",
    "PASSENGERS",
    "FREIGHT",
    "MAIL",
    "DISTANCE",
]

In [0]:
if "sub_df" not in locals():
    sub_df = df.copy()
    del df
    sub_df = sub_df[needed_columns]
sub_df

,UNIQUE_CARRIER_NAME,CARRIER_NAME,YEAR,MONTH,QUARTER,PASSENGERS,FREIGHT,MAIL,DISTANCE
0,Independence Air,Atlantic Coast Airlines,2003,8,3,0.0,0.0,0.0,464.0
1,Federal Express Corporation,Federal Express Corporation,2003,10,4,0.0,2625140.0,0.0,1836.0
2,Delta Air Lines Inc.,Delta Air Lines Inc.,2003,9,3,0.0,29451.0,3324.0,3469.0
3,Cape Air,Cape Air,2003,10,4,0.0,0.0,27935.0,30.0
4,Federal Express Corporation,Federal Express Corporation,2003,12,4,0.0,2893520.0,0.0,570.0
...,...,...,...,...,...,...,...,...,...
6737037,Delta Air Lines Inc.,Delta Air Lines Inc.,2003,3,1,81789.0,462199.0,96084.0,403.0
6737038,Delta Air Lines Inc.,Delta Air Lines Inc.,2003,10,4,82358.0,316493.0,138372.0,403.0
6737039,Delta Air Lines Inc.,Delta Air Lines Inc.,2003,3,1,82409.0,409981.0,198281.0,406.0
6737040,Delta Air Lines Inc.,Delta Air Lines Inc.,2003,3,1,85908.0,284806.0,404759.0,406.0


In [0]:
# correcting the dtypes

sub_df = sub_df.astype(
    {
        "PASSENGERS": "int", 
        "FREIGHT": "int", 
        "MAIL": "int", 
        "DISTANCE": "int"
    }
)

In [0]:
# checking for null values

sub_df.isna().sum()

UNIQUE_CARRIER_NAME    798
CARRIER_NAME           798
YEAR                     0
MONTH                    0
QUARTER                  0
PASSENGERS               0
FREIGHT                  0
MAIL                     0
DISTANCE                 0
dtype: int64

In [0]:
sub_df = sub_df.dropna(axis=0)

# be sure that there are no nans
assert sub_df.isna().sum().sum() == 0

## 3-2 removing outliers

In [0]:
grouped_carrier = sub_df.groupby("UNIQUE_CARRIER_NAME").count()["YEAR"]


In [0]:
# keeping only carriers with more than 10000 total flights
list_of_valid_carriers = grouped_carrier[grouped_carrier > 10000].index.to_numpy()
sub_df = sub_df[sub_df["UNIQUE_CARRIER_NAME"].isin(list_of_valid_carriers)]

In [0]:

sub_df["PASSENGERS_x_DISTANCE"] = sub_df["PASSENGERS"] * sub_df["DISTANCE"]
sub_df["FREIGHT_x_DISTANCE"] = sub_df["FREIGHT"] * sub_df["DISTANCE"]
sub_df["MAIL_x_DISTANCE"] = sub_df["MAIL"] * sub_df["DISTANCE"]


In [0]:
sub_df

,UNIQUE_CARRIER_NAME,CARRIER_NAME,YEAR,MONTH,QUARTER,PASSENGERS,FREIGHT,MAIL,DISTANCE,PASSENGERS_x_DISTANCE,FREIGHT_x_DISTANCE,MAIL_x_DISTANCE
0,Independence Air,Atlantic Coast Airlines,2003,8,3,0,0,0,464,0,0,0
1,Federal Express Corporation,Federal Express Corporation,2003,10,4,0,2625140,0,1836,0,4819757040,0
2,Delta Air Lines Inc.,Delta Air Lines Inc.,2003,9,3,0,29451,3324,3469,0,102165519,11530956
4,Federal Express Corporation,Federal Express Corporation,2003,12,4,0,2893520,0,570,0,1649306400,0
5,Federal Express Corporation,Federal Express Corporation,2003,12,4,0,1199260,0,1606,0,1926011560,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6737037,Delta Air Lines Inc.,Delta Air Lines Inc.,2003,3,1,81789,462199,96084,403,32960967,186266197,38721852
6737038,Delta Air Lines Inc.,Delta Air Lines Inc.,2003,10,4,82358,316493,138372,403,33190274,127546679,55763916
6737039,Delta Air Lines Inc.,Delta Air Lines Inc.,2003,3,1,82409,409981,198281,406,33458054,166452286,80502086
6737040,Delta Air Lines Inc.,Delta Air Lines Inc.,2003,3,1,85908,284806,404759,406,34878648,115631236,164332154


In [0]:
# TEST
tt = sub_df.groupby("UNIQUE_CARRIER_NAME").sum()
tt.sort_values(by="PASSENGERS_x_DISTANCE")

,YEAR,MONTH,QUARTER,PASSENGERS,FREIGHT,MAIL,DISTANCE,PASSENGERS_x_DISTANCE,FREIGHT_x_DISTANCE,MAIL_x_DISTANCE
UNIQUE_CARRIER_NAME,,,,,,,,,,
ABX Air Inc,136437126,450571,172970,0,21693055012,0,41572950,0,20005698628726,0
Village Aviation,23027886,77573,29737,0,32099038,39884031,933576,0,2753940237,3486836383
Reliant Airlines,20990119,69531,26457,0,30389682,21659,5934389,0,17490468186,12408679
Northern Air Cargo Inc.,28280098,96670,36582,0,875042662,536660101,5592354,0,307032116641,232899494294
Kitty Hawk Aircargo,57902925,188710,72425,0,4739190304,924818,18412791,0,3493652168102,2108585040
...,...,...,...,...,...,...,...,...,...,...
Continental Air Lines Inc.,666357758,2139821,820592,1051041617,12179255845,4331042363,505751473,1394387385442,35947034381929,6338026471897
Northwest Airlines Inc.,827104821,2687567,1033929,1329165695,21309608362,7158341441,677991181,1718277928631,88714363291357,9992747616804
Delta Air Lines Inc.,864171030,2797787,1076103,2356677142,24722306665,13997708282,574549790,2391668084420,57673370941304,15930202923683


In [0]:
# tt_df["date"] = 
"{}-{}-01".format(sub_df["YEAR"].astype("str"), sub_df["MONTH"].astype("str"))

'0          2003\n1          2003\n2          2003\n4          2003\n5          2003\n           ... \n6737037    2003\n6737038    2003\n6737039    2003\n6737040    2003\n6737041    2003\nName: YEAR, Length: 6329734, dtype: object-0           8\n1          10\n2           9\n4          12\n5          12\n           ..\n6737037     3\n6737038    10\n6737039     3\n6737040     3\n6737041    10\nName: MONTH, Length: 6329734, dtype: object-01'

In [0]:
grouped_carrier_year_month = sub_df.groupby(["UNIQUE_CARRIER_NAME", "YEAR", "MONTH"]).count()


In [0]:
grouped_carrier_year_month

CARRIER_NAME  QUARTER  PASSENGERS  \
UNIQUE_CARRIER_NAME        YEAR MONTH                                      
ABX Air Inc                2002 10              378      378         378   
                                11              425      425         425   
                                12              422      422         422   
                           2003 1              6090     6090        6090   
                                2              5684     5684        5684   
...                                             ...      ...         ...   
Yute Air Aka Flight Alaska 2003 8              1778     1778        1778   
                                9              1750     1750        1750   
                                10             2226     2226        2226   
                                11             1442     1442        1442   
                                12             2044     2044        2044   

                                       FREIGHT  MAIL  DISTANCE  \
UNIQUE_CARRIER_NAME        YEAR MONTH                            
ABX Air Inc                2002 10         378   378       378   
                                11         425   425       425   
                                12         422   422       422   
                           2003 1         6090  6090      6090   
                                2         5684  5684      5684   
...                                        ...   ...       ...   
Yute Air Aka Flight Alaska 2003 8         1778  1778      1778   
                                9         1750  1750      1750   
                                10        2226  2226      2226   
                                11        1442  1442      1442   
                                12        2044  2044      2044   

                                       PASSENGERS_x_DISTANCE  \
UNIQUE_CARRIER_NAME        YEAR MONTH                          
ABX Air Inc                2002 10                       378   
                                11                       425   
                                12                       422   
                           2003 1                       6090   
                                2                       5684   
...                                                      ...   
Yute Air Aka Flight Alaska 2003 8                       1778   
                                9                       1750   
                                10                      2226   
                                11                      1442   
                                12                      2044   

                                       FREIGHT_x_DISTANCE  MAIL_x_DISTANCE  
UNIQUE_CARRIER_NAME        YEAR MONTH                                       
ABX Air Inc                2002 10                    378              378  
                                11                    425              425  
                                12                    422              422  
                           2003 1                    6090             6090  
                                2                    5684             5684  
...                                                   ...              ...  
Yute Air Aka Flight Alaska 2003 8                    1778             1778  
                                9                    1750             1750  
                                10                   2226             2226  
                                11                   1442             1442  
                                12                   2044             2044  

[7132 rows x 9 columns]

In [0]:
tt = sub_df.groupby(["UNIQUE_CARRIER_NAME", "YEAR", "MONTH"]).count()

In [0]:
tt

CARRIER_NAME  QUARTER  PASSENGERS  \
UNIQUE_CARRIER_NAME        YEAR MONTH                                      
ABX Air Inc                2002 10              378      378         378   
                                11              425      425         425   
                                12              422      422         422   
                           2003 1              6090     6090        6090   
                                2              5684     5684        5684   
...                                             ...      ...         ...   
Yute Air Aka Flight Alaska 2003 8              1778     1778        1778   
                                9              1750     1750        1750   
                                10             2226     2226        2226   
                                11             1442     1442        1442   
                                12             2044     2044        2044   

                                       FREIGHT  MAIL  DISTANCE  \
UNIQUE_CARRIER_NAME        YEAR MONTH                            
ABX Air Inc                2002 10         378   378       378   
                                11         425   425       425   
                                12         422   422       422   
                           2003 1         6090  6090      6090   
                                2         5684  5684      5684   
...                                        ...   ...       ...   
Yute Air Aka Flight Alaska 2003 8         1778  1778      1778   
                                9         1750  1750      1750   
                                10        2226  2226      2226   
                                11        1442  1442      1442   
                                12        2044  2044      2044   

                                       PASSENGERS_x_DISTANCE  \
UNIQUE_CARRIER_NAME        YEAR MONTH                          
ABX Air Inc                2002 10                       378   
                                11                       425   
                                12                       422   
                           2003 1                       6090   
                                2                       5684   
...                                                      ...   
Yute Air Aka Flight Alaska 2003 8                       1778   
                                9                       1750   
                                10                      2226   
                                11                      1442   
                                12                      2044   

                                       FREIGHT_x_DISTANCE  MAIL_x_DISTANCE  
UNIQUE_CARRIER_NAME        YEAR MONTH                                       
ABX Air Inc                2002 10                    378              378  
                                11                    425              425  
                                12                    422              422  
                           2003 1                    6090             6090  
                                2                    5684             5684  
...                                                   ...              ...  
Yute Air Aka Flight Alaska 2003 8                    1778             1778  
                                9                    1750             1750  
                                10                   2226             2226  
                                11                   1442             1442  
                                12                   2044             2044  

[7132 rows x 9 columns]